In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
import requests
import time

In [9]:
# 현재 시간
def _get_datetime_():
    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H:00")

    return now

In [63]:
pollution_all

[{'so2Value': '0.002',
  'coValue': '0.5',
  'pm10Value': '45',
  'no2Value': '0.026',
  'stationName': '음성읍',
  'o3Value': '0.006'},
 {'so2Value': '0.001',
  'coValue': '0.3',
  'pm10Value': '-',
  'no2Value': '0.013',
  'stationName': '소이면',
  'o3Value': '0.012'},
 {'so2Value': '0.002',
  'coValue': '0.6',
  'pm10Value': '30',
  'no2Value': '0.022',
  'stationName': '영동읍',
  'o3Value': '0.006'},
 {'so2Value': '0.003',
  'coValue': '0.3',
  'pm10Value': '34',
  'no2Value': '0.019',
  'stationName': '황간면',
  'o3Value': '0.014'},
 {'so2Value': '0.002',
  'coValue': '0.4',
  'pm10Value': '54',
  'no2Value': '-',
  'stationName': '도안면',
  'o3Value': '0.007'},
 {'so2Value': '0.002',
  'coValue': '0.5',
  'pm10Value': '45',
  'no2Value': '0.019',
  'stationName': '증평읍',
  'o3Value': '0.008'},
 {'so2Value': '0.002',
  'coValue': '0.4',
  'pm10Value': '41',
  'no2Value': '0.015',
  'stationName': '보은읍',
  'o3Value': '0.008'},
 {'so2Value': '0.002',
  'coValue': '0.5',
  'pm10Value': '36',
  '

In [71]:
def get_pollution() :
    key = 'BVQCCUgpx/b8HvRWaOf+lXv7wzOcIMh6bXw/JyBoqDbstzhj6bDJhPY7UW+H296mK5kLc/jmUs4+kpuAUA7t+g=='
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
        
    params ={
        'serviceKey' : key, 
        'returnType' : 'json', 
        'numOfRows' : 10000, 
        'sidoName' : "전국",
        }

    res = requests.get(url, params=params)
    result = res.json()

    response = result.get("response", None)
    if response is None: 
        raise Exception("Response가 비어있음")

    body = response.get("body", None)
    if body is None: 
        raise Exception("Body가 비어있음")

    total_count = body.get("totalCount", None)
    if total_count is None: 
        raise Exception("가져온 아이템수가 0임")

    items = body.get("items", None)
    if items is None: 
        raise Exception("아이템이 한개도 없음")


    pollution_all = {}
    for item in items:
        station = "NULL"
        for key, value in item.items():
            poll = {}
            if key == "stationName":
                station = value
                
            if  key == "so2Value" or key == "coValue" or key == "pm10Value" or key == "no2Value" or key == "o3Value":
                poll[key] = value
                
        pollution_all[station] = poll
    
    # 별칭 담겨있는 텍스트 파일 읽어오기
    with open("station_alias_dic.txt", "r") as f:
        name_tags = f.readlines()
    
    # 모두 모을 데이터 프레임 생성
    all_df = pd.DataFrame()
    
    
    for station, pollution in pollution_all.items():
        df_temp = pd.DataFrame(pollution, index = [0])
        
        station_nametag = ""
        # 측정소명의 별칭 가져오기
        for tag in name_tags:
            tag = tag.strip()
            if station in tag:
                station_nametag = tag.split(":")[1]
        
        # 컬럼명 변경
        df_temp.columns = [key+station_nametag for key in pollution.keys()]
        
        # 데이터프레임 더해주기
        all_df = pd.concat([all_df, df_temp], axis = 1)
        
        return all_df


In [72]:
df = get_pollution()

NameError: free variable 'station_nametag' referenced before assignment in enclosing scope

In [66]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [10]:
# 에어코리아 지역별 호출
def get_pollution_API() -> DataFrame:
    key = 'BVQCCUgpx/b8HvRWaOf+lXv7wzOcIMh6bXw/JyBoqDbstzhj6bDJhPY7UW+H296mK5kLc/jmUs4+kpuAUA7t+g=='
    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
    
    params ={
        'serviceKey' : key, 
        'returnType' : 'json', 
        'numOfRows' : 10000, 
        'sidoName' : "전국",
        }

    res = requests.get(url, params=params)
    result = res.json()

    response = result.get("response", None})
    if response is None: return None
    
    body = response.get("body", None)
    if body is None: return None
    
    total_count = body.get("total_count", None)
    if total_count is None: return None
    
    items = body.get("items", None)
    if items is None: return None
    
    
    pollution_all = {}
    for item in items:
        station = "NULL"
        for key, value in item.items():
            poll = {}
            if key == "stationName":
                station = value
                
            if  key == "so2Value" or key == "coValue" or key == "pm10Value" or key == "no2Value" or key == "o3Value":
                poll[key] = value
                
        pollution_all[station] = poll

    # 별칭 담겨있는 텍스트 파일 읽어오기
    with open("station_alias_dic.txt", "r") as f:
        name_tags = f.readlines()
    
    # 모두 모을 데이터 프레임 생성
    all_df = pd.DataFrame()

    for i in range(0, len(pollution_all)):
        # 각 측정소 마다 데이터프레임 만들면서 컬럼명에 별칭 추가
        df_temp = pd.DataFrame(pollution_all[i]["Values"], index = [0])
        
        #측정소명 가져오기
        station = pollution_all[i]["Station"] 

        # 측정소명의 별칭 가져오기
        for tag in name_tags:
            tag = tag.strip()
            if station in tag:
                station_nametag = tag.split(":")[1]
        
        # 컬럼명 변경
        df_temp.columns = [key+station_nametag for key in pollution_all[i]["Values"].keys()]
        
        # 데이터프레임 더해주기
        all_df = pd.concat([all_df, df_temp], axis = 1)

    return all_df


In [11]:
# 에어코리아 정보 호출
def get_all_pollution() -> DataFrame:
    # 지금 시간 가져오기
    time_now = _get_datetime_()

    # 빈 데이터프레임 생성
    final_dataframe = pd.DataFrame({"datetime": [time_now]})

    # 불러올 관측소 시도명 리스트
    area = ["인천","서울","경기"]

    # 돌아가면서 3개의 시도를 함수에 넣기
    for SidoName in area:
        # api 연결해서 데이터 가져와서 딕셔너리 형식으로 저장
        result_df = get_pollution_API(sidoName=SidoName)
        # 결과물 최종 데이터프레임에 저장
        final_dataframe = pd.concat([final_dataframe, result_df], axis = 1)
        
    # 지금 컬럼이 object로 되어 있어서 string으로 변경
    final_dataframe.set_index('datetime',inplace=True)
    final_dataframe.replace("-", "0", inplace=True)
    try:
        final_dataframe = final_dataframe.astype('float')
    except Exception as e:
        print(e)
    
    final_dataframe = final_dataframe.reset_index()

    return final_dataframe


In [12]:
df = get_all_pollution()

In [13]:
df.head()

,datetime,SO2_incheon_1,CO_incheon_1,O3_incheon_1,NO2_incheon_1,PM10_incheon_1,SO2_incheon_16,CO_incheon_16,O3_incheon_16,NO2_incheon_16,...,SO2_gyeongi_23,CO_gyeongi_23,O3_gyeongi_23,NO2_gyeongi_23,PM10_gyeongi_23,SO2_gyeongi_24,CO_gyeongi_24,O3_gyeongi_24,NO2_gyeongi_24,PM10_gyeongi_24
0,2023-02-16 17:00,0.001,0.5,0.054,0.004,29.0,0.001,0.2,0.048,0.004,...,0.002,0.4,0.047,0.013,22.0,0.001,0.3,0.045,0.015,32.0


In [38]:
for col in df.columns:
    print(col, end=" ")

datetime SO2_incheon_1 CO_incheon_1 O3_incheon_1 NO2_incheon_1 PM10_incheon_1 SO2_incheon_16 CO_incheon_16 O3_incheon_16 NO2_incheon_16 PM10_incheon_16 SO2_incheon_17 CO_incheon_17 O3_incheon_17 NO2_incheon_17 PM10_incheon_17 SO2_incheon_18 CO_incheon_18 O3_incheon_18 NO2_incheon_18 PM10_incheon_18 SO2_incheon_8 CO_incheon_8 O3_incheon_8 NO2_incheon_8 PM10_incheon_8 SO2_incheon_5 CO_incheon_5 O3_incheon_5 NO2_incheon_5 PM10_incheon_5 SO2_incheon_3 CO_incheon_3 O3_incheon_3 NO2_incheon_3 PM10_incheon_3 SO2_incheon_4 CO_incheon_4 O3_incheon_4 NO2_incheon_4 PM10_incheon_4 SO2_incheon_10 CO_incheon_10 O3_incheon_10 NO2_incheon_10 PM10_incheon_10 SO2_incheon_11 CO_incheon_11 O3_incheon_11 NO2_incheon_11 PM10_incheon_11 SO2_incheon_12 CO_incheon_12 O3_incheon_12 NO2_incheon_12 PM10_incheon_12 SO2_incheon_2 CO_incheon_2 O3_incheon_2 NO2_incheon_2 PM10_incheon_2 SO2_incheon_6 CO_incheon_6 O3_incheon_6 NO2_incheon_6 PM10_incheon_6 SO2_incheon_13 CO_incheon_13 O3_incheon_13 NO2_incheon_13 PM10_i

In [42]:
# 기상청 지역별 호출
def get_weather_API(station) -> DataFrame:
    url = f"https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm=202211300900&stn={station_dictionary[station]}&help=0&authKey=jvwEqhhGTIW8BKoYRhyFkQ&dataType=JSON"

    response = requests.get(url)
    lines = response.text.split("\n")

    datas = []
    for i in range(2, len(lines)-2):
        if i < 4: # Columns
            lines[i] = lines[i].replace("#", " ")
            lines[i] = lines[i].replace("-", " ")
        line = lines[i] + '\n'
        datas.append(line)

    with open("temp.csv", 'w') as f:
        for line in datas:
            f.write(line)

    df = pd.read_csv("temp.csv", sep="\s+")
    df = df.drop(index=0, axis=0)

    columns_need = ['STN', 'TA', "RN", 'WS', 'WD', 'HM', "PV", 'TD', 'PA', 
                    "PS", "SS", 'SI', 'SD.2', 'CA', 'CA.1', 'CH', 'VS', 'TS', 'WW']
    df = df[columns_need]
    
    # 컬럼명 변경 - 테이블에 맞춰서
    df.columns = ['STN', 'ta', "rn", 'ws', 'wd', 'hm', "pv", 'td', 
                  'pa', "ps", "ss", 'icsr', 'dsnw', 'dc10Tca', 'dc10LmcsCa', 
                  'IcsCh', 'vs', 'ts', 'WW']
    
    # 컬럼명 포맷팅 ("WW" 빼고)
    names = {'102': 'bangryeong', '112': 'incheon', '201': 'ganghwado', 
              '203': 'icheon', '99' : 'paju', '98' : 'dongducheon', 
              '119' : 'suwon', '202' : 'yangpyeong', '108' : 'seoul'}
    
    number = {'102': '0', '112': '1', '201': '2', 
              '203': '3', '99' : '4', '98' : '5', 
              '119' : '6', '202' : '7', '108' : '8'}
    
    # 관측소명 삭제
    station = df["STN"].unique()[0]
    df.drop("STN", axis = 1, inplace = True)
    
    # 컬럼명 변경 및 포맷팅
    for col_name in df.columns:
        if col_name == "datetime" or col_name == "WW":
            continue
        new_col = col_name + "_" + names[station]
        df.rename(columns = {col_name : new_col + "_" + number[station]}, 
                  inplace = True)
        
    # onehotencode weather codes
    dummy = pd.get_dummies(df["WW"], prefix = "dmstMtphNo")
    df = pd.concat([df, dummy], axis = 1)
    df.drop("WW", axis=1, inplace=True)
            
    return df


In [43]:
# 불러올 관측소 시도명 리스트
station_dictionary = {'서울' : '108',
                  '동두천':'98', 
                  '수원':'119', 
                  "양평":'202', 
                  '이천':'203', 
                  '파주':'99', 
                  '백령도':"102", 
                  '인천':"112",
                  '강화도':'201'}
    
def get_all_weather():
    # 지금 시간 가져오기
    time_now = _get_datetime_()

    # 빈 데이터프레임 생성
    final_dataframe = pd.DataFrame()

    # 돌아가면서 3개의 시도를 함수에 넣기
    for station in station_dictionary:
        # api 연결해서 데이터 가져와서 딕셔너리 형식으로 저장
        result_df = get_weather_API(station=station)
        # 결과물 최종 데이터프레임에 저장
        final_dataframe = pd.concat([final_dataframe, result_df], axis = 1)
        
    with open("model_features.txt", "r")as f:
        table_columns = f.read()
    
    for col in final_dataframe.columns:
        if col not in table_columns:
            final_dataframe.drop(col, axis=1, inplace=True)
    
    # # 지금 컬럼이 object로 되어 있어서 string으로 변경
    # final_dataframe.set_index('datetime',inplace=True)
    # final_dataframe.replace("-", "0", inplace=True)
    # try:
    #     final_dataframe = final_dataframe.astype('float')
    # except Exception as e:
    #     print(e)
    
    final_dataframe = final_dataframe.reset_index()

    return final_dataframe


In [31]:
w_df1 = get_weather_API("서울")

In [32]:
w_df1.head()

,ta_seoul_8,rn_seoul_8,ws_seoul_8,wd_seoul_8,hm_seoul_8,pv_seoul_8,td_seoul_8,pa_seoul_8,ps_seoul_8,ss_seoul_8,icsr_seoul_8,dsnw_seoul_8,dc10Tca_seoul_8,dc10LmcsCa_seoul_8,vs_seoul_8,ts_seoul_8
1,-6.6,-9.0,4.1,27,51.0,1.9,-15.0,1022.3,1033.5,1.0,0.47,-9.0,0,0,2000,-2.2


In [44]:
weather_df = get_all_weather()

KeyError: "['dmstMtphNo_-'] not found in axis"

In [34]:
len(weather_df.columns)

142

In [ ]:
for col in weather_df.columns:
    print(c0la)

In [35]:
all_columns = []
for col in df.columns:
    all_columns.append(col)
    
for col in weather_df.columns:
    all_columns.append(col)

In [36]:
for col in all_columns:
    print(col, end=" ")

datetime SO2_incheon_1 CO_incheon_1 O3_incheon_1 NO2_incheon_1 PM10_incheon_1 SO2_incheon_16 CO_incheon_16 O3_incheon_16 NO2_incheon_16 PM10_incheon_16 SO2_incheon_17 CO_incheon_17 O3_incheon_17 NO2_incheon_17 PM10_incheon_17 SO2_incheon_18 CO_incheon_18 O3_incheon_18 NO2_incheon_18 PM10_incheon_18 SO2_incheon_8 CO_incheon_8 O3_incheon_8 NO2_incheon_8 PM10_incheon_8 SO2_incheon_5 CO_incheon_5 O3_incheon_5 NO2_incheon_5 PM10_incheon_5 SO2_incheon_3 CO_incheon_3 O3_incheon_3 NO2_incheon_3 PM10_incheon_3 SO2_incheon_4 CO_incheon_4 O3_incheon_4 NO2_incheon_4 PM10_incheon_4 SO2_incheon_10 CO_incheon_10 O3_incheon_10 NO2_incheon_10 PM10_incheon_10 SO2_incheon_11 CO_incheon_11 O3_incheon_11 NO2_incheon_11 PM10_incheon_11 SO2_incheon_12 CO_incheon_12 O3_incheon_12 NO2_incheon_12 PM10_incheon_12 SO2_incheon_2 CO_incheon_2 O3_incheon_2 NO2_incheon_2 PM10_incheon_2 SO2_incheon_6 CO_incheon_6 O3_incheon_6 NO2_incheon_6 PM10_incheon_6 SO2_incheon_13 CO_incheon_13 O3_incheon_13 NO2_incheon_13 PM10_i

In [37]:
len(all_columns)

733

In [39]:
len(weather_df.columns)

142

In [40]:
len(df.columns)

591